# 최적화

- 모델을 학습 하는 과정은 반복적인 과정을 거침.
- epoch라고 부르는 각 반복 단계에서 모델은 출력을 추정하고 추정값과 정답 사이의 손실을 계산하고 
  매개변수에 대한 오루의 도함수를 수집한 뒤 경사하강법을 이용하여 이 파라미터들을 최적화 함.

## Pre_requisite code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameter

- Hyperparameter는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수임.
- 서로 다른 Hyperparameter 값은 모델 학습과 수렴율에 영향을 미침.

### 학습시 다음과 같은 Hyperparameter를 정의함
- epoch: 데이터셋을 반복하는 횟수
- batch size: 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- learnig rate: 각 batch/epoch에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습속도가 느려지고 
  값이 크면 학습중 예측 불가능한 동작이 발생가능함.
  

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계 

Hyperparameter를 설정한 뒤 최적화 단계를 통해 모델을 학습하고 최적화 할 수 있음.
최적화 단계의 각 반복을 epoch라고 함.

하나의 eopch는 다음과 같이 두 부분을 구성됨.
- training loop: train dataset을 반복하여 최적의 매개변수로 수렴
- validation/test loop: 모델성능이 개선되고 있는지 test dataset을 반복

## 전체 구현

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # prediction과 loss 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100==0:
            loss, current = loss.item(), batch*len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:5d}]')
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f" Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    print("Done!")

Epoch 1
----------------------------------
loss: 2.310322 [    0/60000]
loss: 2.293511 [ 6400/60000]
loss: 2.276363 [12800/60000]
loss: 2.266679 [19200/60000]
loss: 2.253005 [25600/60000]
loss: 2.216107 [32000/60000]
loss: 2.226394 [38400/60000]
loss: 2.190170 [44800/60000]
loss: 2.196546 [51200/60000]
loss: 2.155022 [57600/60000]
 Test Error: 
 Accuracy: 41.3%, Avg loss: 2.157306

Done!
Epoch 2
----------------------------------
loss: 2.168975 [    0/60000]
loss: 2.155646 [ 6400/60000]
loss: 2.105287 [12800/60000]
loss: 2.119674 [19200/60000]
loss: 2.077907 [25600/60000]
loss: 2.008250 [32000/60000]
loss: 2.041592 [38400/60000]
loss: 1.964770 [44800/60000]
loss: 1.972336 [51200/60000]
loss: 1.895693 [57600/60000]
 Test Error: 
 Accuracy: 56.5%, Avg loss: 1.901376

Done!
Epoch 3
----------------------------------
loss: 1.929987 [    0/60000]
loss: 1.896259 [ 6400/60000]
loss: 1.795974 [12800/60000]
loss: 1.830253 [19200/60000]
loss: 1.732203 [25600/60000]
loss: 1.671159 [32000/60000]
l